# Import Required Libraries
Import the necessary libraries, including pathlib, matplotlib, os, and cgshop2025_pyutils.

   >[!NOTE]
   >
   >Notebook requires the `ipywidgets` and `cgshop2025_pyutils` library for python, it is necessary to install with next line command:
   >
   >```bash
   >pip install ipywidgets
   >pip install cgshop2025_pyutils
   >```

In [ ]:
# Import the necessary libraries
from pathlib import Path
from matplotlib import pyplot as plt
import ipywidgets as widgets

# Import specific classes and functions from cgshop2025_pyutils
from cgshop2025_pyutils import (
    DelaunayBasedSolver,
    InstanceDatabase,
    ZipSolutionIterator,
    ZipWriter,
    verify,
    visualization
)

# Locate the Instances
Locate the instances using the InstanceDatabase class from cgshop2025_pyutils.

In [ ]:
# Locate the instances using the InstanceDatabase class from cgshop2025_pyutils
idb = InstanceDatabase("example_instances/")

# Display the number of instances found
print(f"Number of instances found: {sum(1 for e in idb)}")

# Delete Existing Solution Zip File
Check if the solution zip file already exists and delete it if it does.

In [ ]:
# Check if the solution zip file already exists and delete it if it does
if Path("example_solutions.zip").exists():
    Path("example_solutions.zip").unlink()
    print("Existing solution zip file deleted.")
else:
    print("No existing solution zip file found.")

# Compute Solutions for All Instances
Compute solutions for all instances using the provided DelaunayBasedSolver and store them in a list.

In [ ]:
# Compute solutions for all instances using the provided DelaunayBasedSolver and store them in a list
solutions = []  # Initialize an empty list to store solutions

# Iterate over each instance in the InstanceDatabase
for instance in idb:
    uid = instance.instance_uid  # Get the unique identifier for the instance
    points_x = instance.points_x  # Get the x-coordinates of the points in the instance
    points_y = instance.points_y  # Get the y-coordinates of the points in the instance
    num_points = instance.num_points  # Get the number of points in the instance
    
    # Print instance details for debugging purposes
    print(f"Instance UID: {uid}")
    print(f"Points X: {points_x}")
    print(f"Points Y: {points_y}")
    print(f"Number of Points: {num_points}")
    
    # Create a solver object for the instance using DelaunayBasedSolver
    solver = DelaunayBasedSolver(instance)
    
    # Compute the solution for the instance
    solution = solver.solve()
    
    # Append the computed solution to the solutions list
    solutions.append(solution)


# Write Solutions to Zip File
Write the computed solutions to a new zip file using the ZipWriter class from cgshop2025_pyutils.

In [ ]:
# Write the computed solutions to a new zip file using the ZipWriter class from cgshop2025_pyutils
with ZipWriter("example_solutions.zip") as zw:
    for solution in solutions:
        zw.add_solution(solution)
    print("Solutions written to example_solutions.zip")

Run below code to update with the selected UID 

# Plot Solutions
Plot the solutions using matplotlib and the visualization module from cgshop2025_pyutils.

In [ ]:
# Plot Solutions

# Get first solution for interactive controls
uid_found = False
sol_edges = []
for solution in ZipSolutionIterator("example_solutions.zip"):
    sol_edges = solution.edges
    break

# Get input
for instance in idb:
    first_instance = instance.instance_uid
    break

uid_list = widgets.Dropdown(
    options = [instance.instance_uid for instance in idb],
    value = None,
    description = 'UID'
)

edges_list = widgets.Dropdown(
    options = {str(value):value for value in sol_edges},
    value = None,
    description = 'Edge'
)

def interactive_plot_solutions(sel_uid,sel_edge):
    # Init plot
    plt.ioff()
    fig,axs = plt.subplots()

    #  Find instance with uid
    uid_found = False
    for instance in idb:
        if(instance.instance_uid == sel_uid):
            uid_found = True
            points_x = instance.points_x
            points_y = instance.points_y
            # Plot instance points
            #visualization.plot_instance(axs,instance) 
            break

    if uid_found == False : 
        print("uid not found")
        #exit()instance_selected = 0
    
    # Find solution with iud
    uid_found = False
    sol_edges = []
    for solution in ZipSolutionIterator("example_solutions.zip"):
        if(solution.instance_uid == sel_uid):
            uid_found = True
            sol_edges = solution.edges
    
    edges_list.options = {str(value):value for value in sol_edges}

    if uid_found == False : 
        print("uid not found")
        #exit()
    
    # Plot the edges of the solution
    for edge in sol_edges:
        #axs.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], 'r-')
        # TODO: Logic to higlight in blue the selected edge in code block above
        if(sel_edge == edge):
            axs.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], color='blue')
        else:
            axs.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], 'r-')
    
    # Set titles and labels for the subplots
    axs.set_title("Instance Points and Solution Edges")
    axs.set_xlabel("X Coordinates")
    axs.set_ylabel("Y Coordinates")

    # Set the main title for the figure
    plt.suptitle("Instance Visualization and Solution")

    # Display the plot with the instance and its solution
    if uid_found:
        display(fig)
    fig.clear()
    axs.clear()

widgets.interact(interactive_plot_solutions, sel_uid = uid_list, sel_edge = edges_list)

#[print(f"Triangulation:{tr}") for tr in triang] # Print all triangulations


# Calculate and plot triangulations

In [ ]:

# Get first solution for interactive controls
uid_found = False
sol_edges = []
for solution in ZipSolutionIterator("example_solutions.zip"):
    sol_edges = solution.edges
    break

uid_list = widgets.Dropdown(
    options = [instance.instance_uid for instance in idb],
    value = None,
    description = 'UID'
)
triang = []
triang_list = widgets.Dropdown(
    options = {str(value):value for value in triang},
    value = None,
    description = 'Triangulation'
)

triang = []

prev_uid = None
def interactive_plot_triangulations(sel_uid,sel_triang):
    # Init plot
    #plt.ioff()
    global prev_uid
    uid_changed = False
    if sel_uid != prev_uid and prev_uid != None:
        uid_changed = True
        prev_uid = sel_uid
    
    fig,axs = plt.subplots()

    #  Find instance with uid
    uid_found = False
    for instance in idb:
        if(instance.instance_uid == sel_uid):
            uid_found = True
            points_x = instance.points_x
            points_y = instance.points_y
            # Plot instance points
            #visualization.plot_instance(axs,instance) 
            break

    if uid_found == False : 
        print("uid not found")
        #exit()instance_selected = 0
    
    # Find solution with iud
    uid_found = False
    sol_edges = []
    for solution in ZipSolutionIterator("example_solutions.zip"):
        if(solution.instance_uid == sel_uid):
            uid_found = True
            sol_edges = solution.edges

    if uid_found == False : 
        print("uid not found")
        #exit()

    triang = [] # Array for triangulations [[e1,e2,e3],[e4,e5,e6], ... [...]]

    for i in range(len(sol_edges)): # Iterate over all solution edges
        e_i = [] # Edges matching first vertex 
        e_j = [] # Edges matching second vertex
        #print(f"e_[{i}]: [{sol_edges[i][0]},{sol_edges[i][1]}] ") # Print the soluction edge to be comparted with rest of the edges
        for j in range(i+1,len(sol_edges)):
            if(sol_edges[i][0] == sol_edges[j][0] or sol_edges[i][0] == sol_edges[j][1]):
                e_i.append(sol_edges[j])

            if(sol_edges[i][1] == sol_edges[j][1] or sol_edges[i][1] == sol_edges[j][0]):
                e_j.append(sol_edges[j])

        #[print(f"e_i: {ei}") for ei in e_i] # Print all matches for edges i
        #[print(f"e_j: {ej}") for ej in e_j] # Print all matches for edges j

        # Compare all vertex in e_i and e_j to find the pair of edges with third vertex coincidence
        for ei in e_i:
            for ej in e_j:
                aux = set(ei) & set(ej) # Intersection operation for sets. Length of the result will be 1 if the list ei and ej shares the value of one of his elements
                if len(aux) == 1: # Check length 
                    triang.append([sol_edges[i],ei,ej]) # Add triangulation found on the list
                    #print(f"Triangulation{sol_edges[i],ei,ej}") # Print triangulation found

    if triang != []:             
        triang_list.options = {str(value):value for value in triang} # Update triangulation list
    if uid_changed:
        triang_list.value = None
    
    if sel_triang != None and not uid_changed:
        for edge in sol_edges:
            axs.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], 'r-')
    
        for edge in list(sel_triang):
            axs.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], color='blue')
            #else:
            #    axs.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], 'r-')

    # Set titles and labels for the subplots
    axs.set_title("Instance Points and Solution Edges")
    axs.set_xlabel("X Coordinates")
    axs.set_ylabel("Y Coordinates")

    # Set the main title for the figure
    #plt.suptitle("Instance Visualization and Solution")

    # Display the plot with the instance and its solution
    if uid_found:
        display(fig)
    fig.clear()
    axs.clear()

widgets.interact(interactive_plot_triangulations, sel_uid = uid_list, sel_triang = triang_list)


# Verify Solutions
Verify the solutions using the verify function from cgshop2025_pyutils and print the results.

In [ ]:
# Verify Solutions

# Verify the solutions using the verify function from cgshop2025_pyutils and print the results
for solution in ZipSolutionIterator("example_solutions.zip"):
   instance = idb[solution.instance_uid]  # Retrieve the instance corresponding to the solution
   result = verify(instance, solution)  # Verify the solution against the instance
   print(f"{solution.instance_uid}: {result}")  # Print the verification result
   assert not result.errors, "Expect no errors."  # Ensure there are no errors in the verification result

Esto es una nueva atualización 